<a href="https://colab.research.google.com/github/open-mmlab/mmpose/blob/dev-1.x/demo/MMPose_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMPose Testing

Let's start!

In [31]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 2.3.0 False
torchvision version: 0.18.0
mmpose version: 1.3.1
cuda version: not available
compiler information: clang 14.0.3


In [32]:
# check video 

import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from tqdm import tqdm

# Load JSON file containing bounding boxes
fname = '/Users/kbonnen/Documents/work-repos/mmpose/vis_results/results_2yo-pebbles.json'
with open(fname) as f:
    data = json.load(f)

# Open video file
video_path = '/Users/kbonnen/Downloads/2yo-pebbles.mov'
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Prepare output video file setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Set codec for output video
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('/Users/kbonnen/Downloads/part-child-bbox.mp4', fourcc, fps, frame_size)

# Set up color map for bounding boxes
colors = plt.cm.get_cmap('hsv', 10)

for this_frame in tqdm(range(total_frames)):
    # Position video to current frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, this_frame)
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Extract bounding boxes for the current frame from the JSON data
    bboxes = [
        frame_data['attributes']['_pred_instances']['attributes']['bboxes']['__ndarray__'][0]
        for frame_data in data['pose2d'][this_frame]
    ]
    bboxes = np.array(bboxes)

    track_ids = [frame_data['attributes']['track_id'] for frame_data in data['pose2d'][this_frame]]
    
    # Create plot for current frame with bounding boxes
    fig, ax = plt.subplots()
    ax.imshow(frame)
    
    for idx, bbox in enumerate(bboxes):
        x1, y1, x2, y2 = bbox
        width, height = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor=colors(idx % 10), facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1, str(track_ids[idx]), color=colors(idx % 10), fontsize=12, weight='bold')

        plt.axis('off')
        canvas = FigureCanvas(fig)
        canvas.draw()

    
    # Convert matplotlib figure to an image format for video output
    buf = canvas.buffer_rgba()
    frame_rgb = np.asarray(buf)
    new_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGBA2BGR)
    outfile = f'/Users/kbonnen/Downloads/tmp/file{this_frame:02d}.png'
    cv2.imwrite(outfile, new_frame)
    plt.close(fig)

# Release video capture resources
cap.release()




/var/folders/6r/q50gj1mj2012fhzq95rw61w80000gn/T/ipykernel_9869/2184555629.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap('hsv', 10)
100%|██████████| 3116/3116 [14:27<00:00,  3.59it/s] 


In [33]:
import os
import subprocess
import glob

os.chdir("/Users/kbonnen/Downloads/tmp/")
subprocess.call([
    'ffmpeg', '-framerate', '60', '-i', 'file%02d.png', '-r', '30', '-pix_fmt', 'yuv420p',
    '2yo-pebbles.mp4'
])
for file_name in glob.glob("*.png"):
    os.remove(file_name)

ffmpeg version 5.1.4 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix='/opt/homebrew/Cellar/ffmpeg@5/5.1.4_6' --datadir='/opt/homebrew/Cellar/ffmpeg@5/5.1.4_6/share/ffmpeg' --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --